In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import pairwise_distances
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/Users/thomazaraujo/Documents/CIn-UFPE/PIBIC/Fuzzy_Clustering-main 2/datasets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
labels = df["Outcome"].values
df.drop("Outcome", axis=1, inplace=True)
dados = df.to_numpy()

In [ ]:
def euclidean_distance_matrix_fast(X):
    sq_norms = np.sum(X ** 2, axis=1, keepdims=True)
    dist_matrix = sq_norms + sq_norms.T - 2 * np.dot(X, X.T)
    dist_matrix = np.sqrt(np.maximum(dist_matrix, 0))  # para evitar valores negativos por erro numérico
    return dist_matrix

def init_membership_matrix(n, k):
    membership_matrix = np.random.rand(n, k) # gera uma matriz inicial aleatória com valores entre 0 e 1
    membership_matrix = membership_matrix / membership_matrix.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return membership_matrix

# %% [markdown]
# ### Inicialização dos medoides

# %% [markdown]
# #### 1. Primeiro Medoide
# Selecione o primeiro medoide $m_1$ como o ponto com a menor distância total para todos os outros pontos no conjunto de dados $X$, com $n$ amostras:
# 
# 
# $m_1 = \arg \min_i \left( \sum_{j=1}^{n} d(x_i, x_j) \right)$
# 
# 
# onde $d(x_i, x_j)$ representa a dissimilaridade entre os pontos $x_i$ e $x_j$.
# 
# #### 2. Próximos Medoides
# Para cada próximo medoide $m_k$, com $k = 2, \dots, c$, encontre o ponto $x$ que maximize a menor distância em relação aos medoides já selecionados. Para cada ponto candidato $x$ (ainda não selecionado como medoide), calcule:
# 
# 
# $\text{dist\_mínima}(x) = \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j)$
# 
# 
# Então, selecione o ponto $x$ com a maior distância mínima como o próximo medoide:
# 
# 
# $m_k = \arg \max_{x \in X \setminus \{m_1, \dots, m_{k-1}\}} \left( \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j) \right)$

# %%
def init_medoids(X, c):
    distances = euclidean_distance_matrix_fast(X) # calcula todas as distâncias entre os pontos uma vez só

    total_distances = np.sum(distances, axis=1) # primeiro medoide: menor soma de distâncias
    first_medoid_idx = np.argmin(total_distances)

    medoids_indices = [first_medoid_idx] # armazena os índices dos medoides

    for _ in range(1, c): # para os outros medoides
        max_min_dist = -np.inf # armazena a distância
        next_medoid_idx = -1 # armazena o índice do medoide escolhido

        for i in range(len(X)):
            if i in medoids_indices: # se o ponto já for um medoide
                continue

            min_dist_to_medoids = np.min(distances[i, medoids_indices]) # calcula a menor distância deste ponto para qualquer medoide já escolhido

            if min_dist_to_medoids > max_min_dist:
                max_min_dist = min_dist_to_medoids
                next_medoid_idx = i

        medoids_indices.append(next_medoid_idx)

    return X[medoids_indices]

# %% [markdown]
# ### Atualização da matriz de pertinência

# %% [markdown]
# Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:
# 
# #### $u_{ik} = [\sum_{l=1}^c(\frac{d(x_k,v_i)}{d(x_k,v_l)})^{\frac{1}{m-1}}]^{-1}$

# %%
def update_membership_matrix(data, medoids, m):
    distance_matrix = pairwise_distances(data, medoids, metric='euclidean') ** 2
    distance_matrix = np.fmax(distance_matrix, np.finfo(np.float64).eps)  # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aqui
    
    inverse_distance_matrix = 1 / distance_matrix
    power = 1 / (m - 1)
    updated_membership_matrix = (inverse_distance_matrix ** power) / np.sum(inverse_distance_matrix ** power, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    
    return updated_membership_matrix

# %% [markdown]
# ### Atualização dos medoides

# %% [markdown]
# Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:
# 
# #### $q = \arg \min_{1 \leq j \leq n} \sum_{k=1}^{n} \left( u_{ik} \right)^m \cdot d(x_j, x_k)$
# 
# Essa fórmula busca, para cada medoide $m_i$, o ponto $p \in C_i$ que minimiza a soma das distâncias dentro do cluster, garantindo que o novo medoide minimize o custo de distância.

# %%
def update_medoids(X, membership_matrix, m=2):
    n, c = X.shape[0], membership_matrix.shape[1]
    distances = euclidean_distance_matrix_fast(X)
    updated_medoids_indices = []

    for i in range(c):  # para cada cluster
        # custo ponderado total para cada possível medoide j
        costs = np.array([
            np.sum((membership_matrix[:, i] ** m) * distances[j, :])
            for j in range(n)
        ])

        # seleciona o ponto com menor custo como novo medoide
        best_medoid_idx = np.argmin(costs)
        updated_medoids_indices.append(best_medoid_idx)

    return X[updated_medoids_indices]

# %% [markdown]
# ### Fuzzy C-Medoids (FCMdd)

# %% [markdown]
# Etapas:
# - Inicialização da matriz de pertinência
# - Inicialização dos medoides
# - Atualização da matriz de pertinência
# - Atualização dos medoides
# 
# Critérios de parada:
# - Convergência (entre os medoides)
# - Número máximo de iterações

# %%
def fcmdd(data, k, m=2, max_iter=1000000):
    n = data.shape[0]
    membership_matrix = init_membership_matrix(n, k)
    medoids = init_medoids(data, k)
    for _ in range(max_iter):
        membership_matrix = update_membership_matrix(data, medoids, m)
        new_medoids = update_medoids(data, membership_matrix)
        if np.array_equal(medoids, new_medoids): # se os medoides não mudaram, para
            break
        medoids = new_medoids
    return medoids, membership_matrix

# %% [markdown]
# ### Índice de Rand Ajustado (IRA)

# %%
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

# %% [markdown]
# ### Simulação de Monte Carlo

# %%
def monte_carlo_fuzzy_simulation(X, true_labels, k, m=2, num_trials=100):
    results = []
    for trial in range(num_trials):
        medoids, membership_matrix = fcmdd(X, k, m)
        predicted_labels = np.argmax(membership_matrix, axis=1)
        rand_idx = indice_rand(true_labels, predicted_labels)
        results.append(rand_idx)
    
    mean_ari = np.mean(results)
    std_ari = np.std(results)
    return mean_ari, std_ari

# %% [markdown]
# ### Definição de parâmetros e execução do método

# %%
k = 2
num_trials = 100
m = 2
mean_rand_index, std_rand_index = monte_carlo_fuzzy_simulation(dados, labels, k, m, num_trials)

print(f"Monte Carlo FCMdd Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {mean_rand_index:.4f}")
print(f"Standard Deviation of Rand Index: {std_rand_index:.4f}")

Monte Carlo FCMdd Clustering Results (100 trials)
Mean Rand Index: 0.0319
Standard Deviation of Rand Index: 0.0000
